<a href="https://colab.research.google.com/github/hozongsien/PyTorchTutorial/blob/master/Matrix_Factorisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Matrix Factorisation

In [0]:
import numpy as np
from scipy.sparse import rand as sprand
import torch
from torch.utils.data import Dataset, DataLoader

## Hyperparameters

In [0]:
RANDOM_SEED = 0
NUM_EPOCHS = 3
LEARNING_RATE = 1e-6
BATCH_SIZE = 1
PRINT_INTERVAL = 1000
K = 20


def set_seed(seed=RANDOM_SEED):
    torch.manual_seed(RANDOM_SEED)
    np.random.seed(seed=RANDOM_SEED)

## Data Preparation

In [0]:
set_seed()

n_e1_features = 1000
n_e2_features = 1000

X = sprand(n_e1_features, n_e2_features, density=0.01, format='csr')
X.data = (np.random.randint(0, 2, size=X.nnz).astype(np.float64))

X = X.toarray() # convert from scipy to np array

In [0]:
class MatrixDataset(Dataset):
    
    def __init__(self, X):
        self.samples = []
        self._init_dataset()


    def __len__(self):
        return len(self.samples)


    def __getitem__(self, idx):
        ((row, col), label) = self.samples[idx]

        row = row.astype('long')
        col = col.astype('long')
        label = label.astype('float32')

        sample = ((row, col), label)
        return sample


    def _init_dataset(self):
        for row, col in zip(*X.nonzero()):
            input = (row, col)
            label = X[row, col]
            self.samples.append((input, label))

In [0]:
dataset = MatrixDataset(X)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

## MF Model

In [0]:
class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_e1_features, n_e2_features, n_factors=20):
        super().__init__()
        self.U1 = torch.nn.Embedding(n_e1_features, n_factors, sparse=True)
        self.U2 = torch.nn.Embedding(n_e2_features, n_factors, sparse=True)
        

    def forward(self, row, col):
        # specific row/col of latent representation of U1/U2
        return (self.U1(row) * self.U2(col)).sum(1)

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

net = MatrixFactorization(n_e1_features, n_e2_features, K).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=LEARNING_RATE)

# Training

In [8]:
losses = []
running_loss = 0.0
for epoch in range(NUM_EPOCHS):
    for i, ((row, col), label) in enumerate(dataloader):
        row = row.to(device)
        col = col.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        prediction = net(row, col)
        loss = criterion(prediction, label)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        running_loss += loss.item()
        if i % (PRINT_INTERVAL) == (PRINT_INTERVAL - 1):
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / i + 1))
            
    running_loss = 0.0

print('Finished Training')

[1,   100] loss: 18.256
[1,   200] loss: 18.914
[1,   300] loss: 18.923
[1,   400] loss: 20.244
[1,   500] loss: 20.762
[1,   600] loss: 21.673
[1,   700] loss: 22.108
[1,   800] loss: 22.388
[1,   900] loss: 21.712
[1,  1000] loss: 21.882
[1,  1100] loss: 22.285
[1,  1200] loss: 21.982
[1,  1300] loss: 22.153
[1,  1400] loss: 21.967
[1,  1500] loss: 22.300
[1,  1600] loss: 22.181
[1,  1700] loss: 22.404
[1,  1800] loss: 22.268
[1,  1900] loss: 22.551
[1,  2000] loss: 22.478
[1,  2100] loss: 22.333
[1,  2200] loss: 22.193
[1,  2300] loss: 22.090
[1,  2400] loss: 22.134
[1,  2500] loss: 22.087
[1,  2600] loss: 21.914
[1,  2700] loss: 21.820
[1,  2800] loss: 21.832
[1,  2900] loss: 21.898
[1,  3000] loss: 22.116
[1,  3100] loss: 22.045
[1,  3200] loss: 22.219
[1,  3300] loss: 22.269
[1,  3400] loss: 22.381
[1,  3500] loss: 22.342
[1,  3600] loss: 22.480
[1,  3700] loss: 22.456
[1,  3800] loss: 22.544
[1,  3900] loss: 22.534
[1,  4000] loss: 22.490
[1,  4100] loss: 22.329
[1,  4200] loss:

## Test

In [10]:
print('U1:', net.U1.weight)
print('U2:', net.U2.weight)

U1: Parameter containing:
tensor([[-1.1258, -1.1524, -0.2505,  ..., -1.6959,  0.5667,  0.7935],
        [ 0.5988, -1.5551, -0.3414,  ...,  0.1124,  0.6407,  0.4412],
        [-0.1022,  0.7924, -0.2897,  ...,  0.7440,  1.5210,  3.4104],
        ...,
        [-1.0818, -1.9676, -2.2639,  ..., -1.2672,  0.1682,  1.0386],
        [ 0.2942,  0.5609, -0.9465,  ...,  1.7208, -1.5395, -0.3518],
        [-0.6871, -0.4509, -2.0768,  ...,  1.6216,  2.1893, -0.4871]],
       device='cuda:0', requires_grad=True)
U2: tensor([[ 1.2675,  0.8782,  0.3519,  ...,  2.1509, -1.3888,  0.4152],
        [ 0.2433, -1.2376, -0.6014,  ..., -0.4035, -2.2144, -1.4043],
        [ 0.5111,  0.4267,  0.7767,  ..., -0.3132, -0.3373,  3.4600],
        ...,
        [ 0.7555, -1.0806, -0.3530,  ...,  1.3783, -0.5061,  0.0167],
        [-1.3877,  2.0509, -1.4594,  ...,  0.2739, -0.6107,  1.2204],
        [ 2.0062,  1.0579,  0.5614,  ..., -0.1737, -0.2559,  1.0347]],
       device='cuda:0', grad_fn=<TransposeBackward0>)
